## Twitter

In [2]:
import numpy as np
import nltk
import tweepy as tw

from sklearn.utils import check_random_state
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from nose.tools import assert_equal, assert_is_instance, assert_true, assert_almost_equal
from numpy.testing import assert_array_equal

import warnings
warnings.filterwarnings("ignore")

### Twitter Application

In [2]:
def connect_twitter_api(cred_file):
    
    # Order: Access Token, Access Token Secret, Consumer Key, Consumer SecretAccess
    with open(cred_file) as fin:
        tokens = [line.rstrip('\n') for line in fin if not line.startswith('#')]

    auth = tw.OAuthHandler(tokens[2], tokens[3])
    auth.set_access_token(tokens[0], tokens[1])

    return tw.API(auth)

In [3]:
api = connect_twitter_api('twitter.cred')
assert_equal(api.get_user('katyperry').screen_name, 'katyperry')
assert_equal(api.get_user('justinbieber').created_at.strftime('%Y %m %d %H %M %S'), '2009 03 28 16 41 22')
assert_equal(api.get_user('BarackObama').name, 'Barack Obama')

### Positive and negative tweets

In [4]:
from nltk.corpus import twitter_samples as tws

In [9]:
def get_pos_neg_tweets(corpus):
    '''
    Creates a training set from twitter_samples corpus.
    
    Parameters
    ----------
    corpus: The nltk.corpus.twitter_samples corpus.
    
    Returns
    -------
    A tuple of (data, targets)
    
    '''
    
    pos_tweets = np.array(corpus.strings('positive_tweets.json'))
    neg_tweets = np.array(corpus.strings('negative_tweets.json'))
    pos_labels = np.ones(pos_tweets.shape[0])
    neg_labels = np.zeros(neg_tweets.shape[0])
    data = np.concatenate((pos_tweets, neg_tweets), axis = 0)
    targets = np.concatenate((pos_labels, neg_labels), axis=0)
    return data, targets

In [10]:
data, targets = get_pos_neg_tweets(tws)

In [11]:
assert_is_instance(data, np.ndarray)
assert_is_instance(targets, np.ndarray)
assert_equal(len(data), 10000)
assert_equal(len(targets), 10000)
assert_array_equal(
    data[:5],
    [
        '#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top '
            'engaged members in my community this week :)',
        '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234'
            ' and we will be able to assist you :) Many thanks!',
        '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing '
            'track. When are you in Scotland?!',
        '@97sides CONGRATS :)',
        'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark '
            'on my fb profile :) in 15 days'
        ]
    )
assert_array_equal(
    data[5000:5005],
    [
        'hopeless for tmr :(',
        "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in "
            "2 months :(",
        '@Hegelbon That heart sliding into the waste basket. :(',
        '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
        'Dang starting next week I have "work" :('
        ]
    )
assert_array_equal(targets[:5000], [1] * 5000)
assert_array_equal(targets[5000:], [0] * 5000)

### Training

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.2, random_state=check_random_state(0)
    )

In [13]:
def train(X_train, y_train, X_test, random_state):
    '''
    Creates a document term matrix and uses Random Forest classifier to make document classifications.
    
    Parameters
    ----------
    X_train: A list of strings.
    y_train: A list of strings.
    X_test: A list of strings.
    random_state: A np.random.RandomState instance.
    
    Returns
    -------
    A tuple of (clf, y_pred)
    clf: A Pipeline instance.
    y_pred: A numpy array.
    '''
    
    tools = [('tf', TfidfVectorizer()), ('rf', RandomForestClassifier(random_state=random_state))]
    clf = Pipeline(tools)
    clf.set_params(tf__stop_words = 'english', \
                   tf__ngram_range=(1,3))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return clf, y_pred

In [14]:
clf, y_pred = train(X_train, y_train, X_test, random_state=check_random_state(0))
score = accuracy_score(y_pred, y_test)
print("RF prediction accuracy = {0:5.1f}%".format(100.0 * score))

RF prediction accuracy =  72.3%


In [15]:
assert_is_instance(clf, Pipeline)
assert_is_instance(y_pred, np.ndarray)
tf = clf.named_steps['tf']
assert_is_instance(tf, TfidfVectorizer)
assert_is_instance(clf.named_steps['rf'], RandomForestClassifier)
assert_equal(tf.stop_words, 'english')
assert_equal(tf.ngram_range, (1, 3))
assert_equal(len(y_pred), len(y_test))
assert_array_equal(y_pred[:10], [0, 1, 1, 0, 1, 0, 0, 0, 1, 1])
assert_array_equal(y_pred[-10:], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0])
assert_almost_equal(score, 0.723)

### User timeline

In [27]:
def get_timeline(user, since_id, max_id):
    '''
    Fetches 20 tweets from "user".
    
    Parameters
    ----------
    user: A string. The ID or screen name of a Twitter user.
    since_id: An int. Returns only statuses with an ID greater than (that is, more recent than) the specified ID.
    max_id: An int. Returns only statuses with an ID less than (that is, older than) or equal to the specified ID..
    
    Returns
    -------
    A list of integers.
    '''
    
    timeline=api.user_timeline(screen_name=user, since_id=since_id, max_id =max_id )
    return timeline

In [29]:
timeline1 = get_timeline('TheDemocrats', since_id =735495811841298432  ,max_id=  837326424117817346 )

In [28]:
timeline2 = get_timeline('GOP', since_id=  734118726795042817, max_id =834928725183586308)

In [30]:
assert_equal(
    [t.id for t in timeline1],
    [837326424117817346,
    837320785794564096,
    837100935457488901,
    837072132290949120,
    837052049317588994,
    837037094686031873,
    837011510446682113,
    836990720640749569,
    836966617632342016,
    836803847762886656,
    836801033133260800,
    836785330455998464,
    836779989068558336,
    836777446821281792,
    836771651694039042,
    836769183375458306,
    836768477943836673,
    836768098275495936,
    836767367107645441,
    836766862566387712]
    )
assert_equal(
    [t.id for t in timeline2],
    [834928725183586308,
    834898073121861635,
    834896078055014401,
    834856145974153216,
    834851413746413568,
    834807922995712000,
    834790862815125504,
    834779274175512576,
    834756446608879618,
    834531775883911168,
    834513243087523841,
    834497491517263872,
    834205062805196800,
    834058475785351168,
    833738748894535680,
    833699482852327425,
    833682386953056256,
    833489605559275520,
    833371582366175233,
    833368197160136706])

### Make predictions

In [31]:
def predict(clf, timeline):
    '''
    Uses a classifier ("clf") to classify each tweet in
    "timeline" as a positive tweet or a negative tweet.
    
    Parameters
    ----------
    clf: A Pipeline instance.
    timeline: A tweepy.models.ResultSet instance.
    
    Returns
    -------
    A numpy array.
    '''
    
    messages = []
    for tweet in timeline:
        messages.append(tweet.text)
    new_tweets = np.array(messages)
    y_pred = clf.predict(new_tweets)
    return y_pred

In [32]:
pred1 = predict(clf, timeline1)
print('{} has {} positive tweets and {} negative tweets.'.format(
    'The Democrats account', (pred1 == 1).sum(), (pred1 == 0).sum()))

The Democrats account has 3 positive tweets and 17 negative tweets.


In [33]:
assert_is_instance(pred1, np.ndarray)
assert_array_equal(
     pred1,
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  
      0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.,  0.]
    )

In [34]:
pred2 = predict(clf, timeline2)
print('{} has {} positive tweets and {} negative tweets.'.format(
    'The GOP account', (pred2 == 1).sum(), (pred2 == 0).sum()))

The GOP account has 14 positive tweets and 6 negative tweets.


In [35]:
assert_is_instance(pred2, np.ndarray)
assert_array_equal(
     pred2,
     [1.,  1.,  1.,  1.,  0. , 1.,  0.,  0.,  1.,  0.,  
      1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., 0.,  1.]
    )